# Cifar10 - exercicio de classificar 3 classes

Fazer classificação de 3 classes usando rede neural convolucional.
Não utilizar o pacote sklearn. Apenas o Keras e o NumPy.

Não utilizar a função de acompanhamento de gráficos durante o treinamento.

Gerar uma figura mosaic que contenha as 5 imagens de classificação correta de menor probabilidade de predição.

Gerar esta figura com o nome: cifar_fig.png

## Importação de bibliotecas

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from sklearn.datasets import load_iris
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torchvision import datasets, transforms
from itertools import count
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## Carregamento do dataset

In [2]:
data = np.load('/etc/jupyterhub/ia368z_2s2017/datasets/cifar10-redux.npz')

In [3]:
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']

In [4]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2000, 3, 32, 32), (2000,), (500, 3, 32, 32), (500,))

In [5]:
X_train.dtype, y_train.dtype, X_test.dtype, y_test.dtype

(dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('int64'))

In [6]:
# Data normalization
X_train = torch.from_numpy(X_train.astype('float32')).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train-3).type(torch.LongTensor)
X_test = torch.from_numpy(X_test.astype('float32')).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test-3).type(torch.LongTensor)
X_train /= 255.
X_test /= 255.


# Split the X_train to validation
X_val = X_train[1500:]
y_val = y_train[1500:]
X_train = X_train[0:1499]
y_train = y_train[0:1499]

In [7]:
def SmallCNN(nclasses,input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',input_shape=input_shape,data_format="channels_first"))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(150))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))
    return model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

In [8]:
# Main cell
model = Net()

# initiate RMSprop optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [9]:
def train(X,y,epoch,b=10):
    model.train()
    #random = np.random.randint(0,len(X),size=(1,b))
    data, target = Variable(X.float()), Variable(y)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, len(data), len(X),100. *len(data) / len(X), loss.data[0]))

def test(X,y):
    model.eval()
    test_loss = 0
    correct = 0
    data, target = Variable(X.float(), volatile=True), Variable(y)
    output = model(data)
    test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(X)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(X),
        100. * correct / len(X)))


In [10]:
for epoch in range(1, 100 + 1):
    train(X_train,y_train,epoch)
    test(X_val,y_val)

Train Epoch: 1 [1499/1499 (100%)]	Loss: 1.103089

Test set: Average loss: 1.1014, Accuracy: 158/500 (32%)

Train Epoch: 2 [1499/1499 (100%)]	Loss: 1.101051

Test set: Average loss: 1.0998, Accuracy: 158/500 (32%)

Train Epoch: 3 [1499/1499 (100%)]	Loss: 1.099288

Test set: Average loss: 1.0987, Accuracy: 158/500 (32%)

Train Epoch: 4 [1499/1499 (100%)]	Loss: 1.097698

Test set: Average loss: 1.0978, Accuracy: 205/500 (41%)

Train Epoch: 5 [1499/1499 (100%)]	Loss: 1.096400

Test set: Average loss: 1.0965, Accuracy: 209/500 (42%)

Train Epoch: 6 [1499/1499 (100%)]	Loss: 1.094867

Test set: Average loss: 1.0948, Accuracy: 204/500 (41%)

Train Epoch: 7 [1499/1499 (100%)]	Loss: 1.092651

Test set: Average loss: 1.0932, Accuracy: 183/500 (37%)

Train Epoch: 8 [1499/1499 (100%)]	Loss: 1.090142

Test set: Average loss: 1.0905, Accuracy: 194/500 (39%)

Train Epoch: 9 [1499/1499 (100%)]	Loss: 1.086797

Test set: Average loss: 1.0865, Accuracy: 210/500 (42%)

Train Epoch: 10 [1499/1499 (100%)]	Lo

Train Epoch: 77 [1499/1499 (100%)]	Loss: 0.835821

Test set: Average loss: 0.9224, Accuracy: 271/500 (54%)

Train Epoch: 78 [1499/1499 (100%)]	Loss: 0.835583

Test set: Average loss: 0.9171, Accuracy: 282/500 (56%)

Train Epoch: 79 [1499/1499 (100%)]	Loss: 0.834767

Test set: Average loss: 0.9214, Accuracy: 272/500 (54%)

Train Epoch: 80 [1499/1499 (100%)]	Loss: 0.831375

Test set: Average loss: 0.9134, Accuracy: 287/500 (57%)

Train Epoch: 81 [1499/1499 (100%)]	Loss: 0.825726

Test set: Average loss: 0.9114, Accuracy: 285/500 (57%)

Train Epoch: 82 [1499/1499 (100%)]	Loss: 0.821347

Test set: Average loss: 0.9132, Accuracy: 278/500 (56%)

Train Epoch: 83 [1499/1499 (100%)]	Loss: 0.818674

Test set: Average loss: 0.9067, Accuracy: 290/500 (58%)

Train Epoch: 84 [1499/1499 (100%)]	Loss: 0.815810

Test set: Average loss: 0.9090, Accuracy: 287/500 (57%)

Train Epoch: 85 [1499/1499 (100%)]	Loss: 0.812489

Test set: Average loss: 0.9074, Accuracy: 288/500 (58%)

Train Epoch: 86 [1499/1499 (

In [11]:
test(X_test,y_test)


Test set: Average loss: 0.9482, Accuracy: 267/500 (53%)

